In [36]:
import pandas as pd
from dateutil import parser

In [22]:
# Quick Configurations

OIL_MONTHLY_CNAME_0 = "date" # Date of the month
OIL_MONTHLY_CNAME_1 = "unit" # how many units are bought
OIL_MONTHLY_CNAME_2 = "price" # total price of all units bought
OIL_MONTHLY_CNAME_3 = "change" # Percentage, looks to be an evolution

OIL_DAILY_CNAME_0 = "date" # Daily date
OIL_DAILY_CNAME_1 = "dcoilwtico" # See onenote - price of oil

# Step 1: Preprocess file to be read in dataframe
- Replace , to ; in oil(india.csv)
- Split into two files (monthly and daily)

In [23]:
# Read the oil(india) file
with open('Pre-processed/oil(india).csv', 'r') as file:
    data = file.readlines()
    data_monthly = [f"{OIL_MONTHLY_CNAME_0};{OIL_MONTHLY_CNAME_1};{OIL_MONTHLY_CNAME_2};{OIL_MONTHLY_CNAME_3}\n"] # Header monthly
    data_daily = [f"{OIL_DAILY_CNAME_0};{OIL_DAILY_CNAME_1}\n"] # Header daily
    
    for idx, row in enumerate(data):
        modified_row = row.replace(',',';')
        
        if(1 <= idx & idx <= 60):
            data_monthly.append(modified_row)
        if(idx > 60):
            # remove last 2 semicolons
            modified_row = ";".join(modified_row.split(";")[:-2])
            data_daily.append(modified_row + "\n")

# Write the modified lines back to the file
with open('Processed/oil(india)_monthly.csv', 'w') as file:
    file.writelines(data_monthly)
with open('Processed/oil(india)_daily.csv', 'w') as file:
    file.writelines(data_daily)

# Step 2: pre process monthly price

- Date formats
- Added price_per_unit

In [42]:
df = pd.read_csv('Processed/oil(india)_monthly.csv', delimiter=';', encoding='ISO-8859-1')

# apr/13 --> apr 2013
df['date'] = df['date'].str.replace('/', ' 20') 

# apr 2013 --> 2013-04
df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: parser.parse(x).strftime('%Y-%m') if isinstance(x, str) else x) 

# add column price per unit
df['price_per_unit'] = round(df['price'] / df['unit'],2)

# Write the modified DataFrame back to the file
df.to_csv('Processed/oil(india)_monthly_clean.csv', index=False)

# Step 3: pre process daily price

- remove NaN values
- check for outliers in price

In [54]:
df = pd.read_csv('Processed/oil(india)_daily.csv', delimiter=';', encoding='ISO-8859-1')
df.dropna(inplace=True)

# Calculate Z-score for the 'dcoilwtico' column
z_scores = (df['dcoilwtico'] - df['dcoilwtico'].mean()) / df['dcoilwtico'].std()

threshold = 2

# Print outliers
outliers = df[abs(z_scores) > threshold]
print("Outliers:")
print(outliers)

df.to_csv('Processed/oil(india)_daily_clean.csv', index=False)

Outliers:
Empty DataFrame
Columns: [date, dcoilwtico]
Index: []
